Подключим необходимые библиотеки

In [31]:
import warnings
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV, ElasticNet, ElasticNetCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, GridSearchCV
warnings.filterwarnings('ignore')

Подготовим функцию для вывода на печать метрик модели:

In [32]:
from sklearn.tree import DecisionTreeRegressor
def print_model_metrics(estimator, y_train, y_pred):
    print(f"Коэффициент детерминации: {estimator.score(x_axis,y_axis)}")
    print(f'MSE: {mean_squared_error(y_train, y_pred)}')
    print(f'RMSE: {mean_squared_error(y_train, y_pred, squared=False)}')
    print(f'MAE: {mean_absolute_error(y_train, y_pred)}')

Загрузим подготовленный в ЛР 1 датасет для классификации

In [33]:
data = pd.read_csv("../data/CSM_snake_case.csv")
data.drop('Movie', axis=1, inplace=True)
data

,Year,Ratings,Genre,Gross,Budget,Screens,Sequel,Sentiment,Views,Likes,Dislikes,Comments,Aggregate Followers
0,2014.0,6.3,8.0,9130.0,4000000.0,45.000000,1.0,0.0,3280543.0,4632.0,425.0,636.0,1.120000e+06
1,2014.0,7.1,1.0,192000000.0,50000000.0,3306.000000,2.0,2.0,583289.0,3465.0,61.0,186.0,1.235000e+07
2,2014.0,6.2,1.0,30700000.0,28000000.0,2872.000000,1.0,0.0,304861.0,328.0,34.0,47.0,4.830000e+05
3,2014.0,6.3,1.0,106000000.0,110000000.0,3470.000000,2.0,0.0,452917.0,2429.0,132.0,590.0,5.680000e+05
4,2014.0,4.7,8.0,17300000.0,3500000.0,2310.000000,2.0,0.0,3145573.0,12163.0,610.0,1082.0,1.923800e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,2015.0,6.4,4.0,1210000.0,50000000.0,66.000000,1.0,4.0,3701061.0,9325.0,641.0,1859.0,3.038193e+06
227,2015.0,5.5,15.0,21000000.0,37000000.0,2815.000000,1.0,13.0,7119456.0,18803.0,1128.0,2290.0,3.038193e+06
228,2015.0,5.4,8.0,10200000.0,35000000.0,2777.000000,1.0,7.0,3450614.0,6823.0,325.0,409.0,3.038193e+06
229,2015.0,5.4,1.0,12300000.0,3000000.0,2209.244344,1.0,10.0,66872.0,400.0,67.0,201.0,3.038193e+06


Выделим вектор исследуемого признака и матрицу объект-признак

In [34]:
x_axis = data.drop('Year', 1).values
y_axis = data['Year'].values

Разобьем выборку на тренировочную и тестовую

In [35]:
X_train, X_test, y_train, y_test = train_test_split(x_axis, y_axis, test_size=0.2, stratify=y_axis)

In [36]:
lr = LinearRegression().fit(X_train, y_train)
print_model_metrics(lr, y_train, lr.predict(X_train))

Коэффициент детерминации: 0.15174448197369939
MSE: 0.15525959846406168
RMSE: 0.39402994615138287
MAE: 0.32428440764952327


Получили неплохое значение коэффициента детерминации, однако ошибки потенциально могут быть уменьшены

In [37]:
parameters = {'max_depth': np.arange(1,15,1),
              'max_features': np.arange(5,x_axis.shape[1]-1,1)}

In [38]:
model = GridSearchCV(DecisionTreeRegressor(random_state=0), parameters).fit(X_train, y_train)
model.best_params_

{'max_depth': 2, 'max_features': 9}

In [39]:
print_model_metrics(model, y_train, model.predict(X_train))

Коэффициент детерминации: 0.1817759043104138
MSE: 0.1585439017504235
RMSE: 0.3981757171782623
MAE: 0.3170878035008041


Значение коэффициента детерминации нормальное.

In [40]:
parameters = {'alpha': np.arange(0.1,2,0.05)}

In [41]:
model = GridSearchCV(Lasso(random_state=0), parameters).fit(X_train, y_train)
model.best_params_

{'alpha': 0.15000000000000002}

In [42]:
print_model_metrics(model, y_train, model.predict(X_train))

Коэффициент детерминации: 0.12783875688062551
MSE: 0.15802992950135034
RMSE: 0.39752978442042597
MAE: 0.3310668439086899


In [43]:
lasso_cv = LassoCV(cv=5, random_state=0).fit(X_train, y_train)

In [44]:
print_model_metrics(lasso_cv, y_train, lasso_cv.predict(X_train))

Коэффициент детерминации: 0.054968483321577
MSE: 0.19202318044612654
RMSE: 0.4382044961500584
MAE: 0.3865640232199795


Обучим и оценим модель Ridge и подберем гиперпараметры

In [45]:
parameters = {'alpha': np.arange(0.1,2,0.05)}

In [46]:
model = GridSearchCV(Ridge(random_state=0), parameters).fit(X_train, y_train)
model.best_params_

{'alpha': 1.9500000000000006}

In [47]:
print_model_metrics(model, y_train, model.predict(X_train))

Коэффициент детерминации: 0.15167593762157328
MSE: 0.1552601194050885
RMSE: 0.39403060719325916
MAE: 0.32434309754907653


In [48]:
ridge_cv = RidgeCV(cv=5).fit(X_train, y_train)

In [49]:
print_model_metrics(ridge_cv, y_train,ridge_cv.predict(X_train))

Коэффициент детерминации: 0.15136790344277407
MSE: 0.15527168103477343
RMSE: 0.3940452778993468
MAE: 0.3245678000386523


Обучим и оценим модель ElasticNet и подберем гиперпараметры

In [50]:
parameters = {'alpha': np.arange(0.1,2,0.05), 'l1_ratio': np.arange(0.1, 1.1, 0.1)}

In [51]:
model = GridSearchCV(ElasticNet(random_state=0), parameters).fit(X_train, y_train)
model.best_params_

{'alpha': 0.20000000000000004, 'l1_ratio': 0.8}

In [52]:
print_model_metrics(model, y_train, model.predict(X_train))

Коэффициент детерминации: 0.1268789375696776
MSE: 0.15811598380102296
RMSE: 0.3976380059816
MAE: 0.3312579940627999


In [53]:
elastic_net_cv = ElasticNetCV(cv=5, random_state=0).fit(X_train, y_train)

In [54]:
print_model_metrics(elastic_net_cv, y_train,elastic_net_cv.predict(X_train))

Коэффициент детерминации: 0.05496848335311466
MSE: 0.1920231804490936
RMSE: 0.43820449615344387
MAE: 0.3865640232468232


### Вывод

В исходных данных не наблюдаются точные линейные зависимости между целевым
и другими признаками, вследствие чего линейные модели уступают в качестве
прогнозирования нелинейным (в частности, дереву решений)